In [3]:
import pandas as pd
import pprint as pp
import plotly.plotly as py
import plotly.graph_objs as go
import datetime as datetime

df = pd.read_csv('../data/AllGunShootings.csv') 

def get_axes():
    
    # insert casualties(casualties AND injured) per incident
    df.insert(value = [kill + inj for kill, inj in zip(df['n_killed'], 
              df['n_injured'])], loc= df.shape[1], column = 'casualties')
    
    # set start values and constant
    old_month = 1
    casualties_num = 0
    incidents_num = 0
    incidents = []
    casualties = []
    columns = df.shape[0]
    weight2018 = 0.2
    standardweight = 0.25
    
    # iterate over rows
    for i in range(278, columns):
        
        # increase if month is the same
        if int(df.iloc[i, 2][5:7]) == old_month:
            casualties_num += df.iloc[i, 19]
            incidents_num += 1
        # gather data if new month is there
        elif(len(incidents) != 12):
            if old_month < 4:
                casualties.append(weight2018 * casualties_num)
                incidents.append(weight2018 * incidents_num)
                old_month = int(df.iloc[i, 2][5:7])
                inj_num = 0
                casualties_num = 0
                incidents_num = 0
            else:
                casualties.append(standardweight * casualties_num)
                incidents.append(standardweight* incidents_num)
                old_month = int(df.iloc[i, 2][5:7])
                inj_num = 0
                casualties_num = 0
                incidents_num = 0
        else:
            if old_month < 4:
                casualties[old_month - 1] += weight2018 * casualties_num
                incidents[old_month - 1] += weight2018 * incidents_num
                old_month = int(df.iloc[i, 2][5:7])
                inj_num = 0
                casualties_num = 0
                incidents_num = 0
            else:
                casualties[old_month - 1] += standardweight * casualties_num
                incidents[old_month - 1] += standardweight * incidents_num
                old_month = int(df.iloc[i, 2][5:7])
                inj_num = 0
                casualties_num = 0
                incidents_num = 0
    
    # append vals for last month
    casualties[old_month - 1] += weight2018 * casualties_num
    incidents[old_month - 1] += weight2018 * incidents_num
    return [casualties, incidents]

months = [datetime.date(2008, i, 1).strftime('%B') for i in range(1,13)]

axes = get_axes()

trace0 = go.Scatter(
    x= months,
    y = axes[1],
    name = 'incidents',
    line = dict(
        shape = 'spline')
)
trace1 = go.Scatter(
    x = months ,
    y = axes[0],
    name = 'casualties',
    line = dict(
        shape = 'spline',
        color = 'rgb(255, 127, 14)')
)

layout0 = dict(title = 'Average amount of incidents per month of the year',
              xaxis = dict(title = 'Month'),
              yaxis = dict(title = 'Incidents'),
              )

layout1 = dict(title = 'Average amount of casualties per month of the year',
              xaxis = dict(title = 'Month'),
              yaxis = dict(title = 'Casualties'),
              )

data0 = [trace0]
data1 = [trace1]
fig = dict(data = data0, layout = layout0)

py.iplot(fig, filename='data_month')